In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime

In [ ]:
#Loading dataset

In [2]:
df=pd.read_csv("Pattuvam_data_till_Aug_2024_corrected.csv")
df

,DATE,TIME,RAW WATER FLOW IN m3/h,CLEAR WATER SUMP LEVEL IN Meter,CLEAR WATER PUMPING FLOW m3/h,TREATED WATER PRODUCTION IN m3/h,REMARKS
0,01/08/2024,01:00,2561.58,2.32,1996.36,2479.10,NaN
1,01/08/2024,02:00,2510.71,2.49,1984.73,2429.87,NaN
2,01/08/2024,03:00,2489.32,2.71,1998.56,2409.16,NaN
3,01/08/2024,04:00,2444.78,2.79,1972.01,2366.06,NaN
4,01/08/2024,05:00,2417.47,2.99,1945.42,2339.63,NaN
...,...,...,...,...,...,...,...
24812,31/12/2021,20:00,2499.51,289,1851.25,2419.03,NaN
24813,31/12/2021,21:00,2477.1,3.06,1878.95,2397.34,NaN
24814,31/12/2021,22:00,2470.76,3.25,1913.29,2391.20,NaN
24815,31/12/2021,23:00,2492.04,3.3,1904.43,2411.80,NaN


In [3]:
df.shape

(24817, 7)

In [ ]:
df[df['REMARKS']=='Maintenance Work at Intake']

In [ ]:
df.isna().sum()

# Making Standardised Date Format

In [ ]:
from dateutil import parser
import pandas as pd

def standardize_date_format(date_str):
    try:
        # Use dateutil's parser to automatically recognize and convert various date formats
        date_obj = parser.parse(date_str)
        # Return the date in DD/MM/YYYY format
        return date_obj.strftime('%d/%m/%Y')
    except (ValueError, TypeError):
        # Handle cases where date_str is not a recognizable date
        return 'Unknown'

# Assuming you have a DataFrame named df with a 'DATE' column
df['Standardized_Date'] = df['DATE'].apply(standardize_date_format)

print("DataFrame with standardized dates:")
df


In [ ]:
df['Standardized_Date'].isna().sum()

In [ ]:
df[df['Standardized_Date']=='Unknown']

# Making Standardised Time Format

In [ ]:
### For time standardisation

def standardize_time(time_str):
 
    time_str = time_str.replace(' ', ':')     #For Replacing the spaces with colons
    
    
        #For cases where time is not in HH:MM format
        
    if len(time_str) == 1:  # Single digit hour
        time_str = f'0{time_str}:00'
    elif len(time_str) == 2:  # Only hour 
        time_str = f'{time_str}:00'
    elif len(time_str) == 4:  # Format like HHMM
        time_str = f'{time_str[:2]}:{time_str[2:]}'
    elif len(time_str) == 5 and ':' not in time_str:  # Format like HHMM
        time_str = f'{time_str[:2]}:{time_str[2:]}'
    
    # Convert to datetime and format as HH:MM
    try:
        return pd.to_datetime(time_str, format='%H:%M', errors='coerce').strftime('%H:%M')
    except ValueError:
        return None  # Return None for unparseable formats

# Apply the function to the TIME column
df['STANDARDIZED_TIME'] = df['TIME'].apply(standardize_time)

In [ ]:
df['STANDARDIZED_TIME'].isna().sum()

In [ ]:
df=df[['Standardized_Date','STANDARDIZED_TIME','RAW WATER FLOW IN m3/h',
       'CLEAR WATER SUMP LEVEL IN Meter', 'CLEAR WATER PUMPING FLOW m3/h',
       'TREATED WATER PRODUCTION IN m3/h', 'REMARKS']]
df

# Data cleaning steps for Standardized Date column

In [ ]:
df.drop(index=range(2136, 2160), inplace=True) #These indexes shows 31/06/2024 data where June month contains only 30 days

In [ ]:
df['Standardized_Date'] = df['Standardized_Date'].str.replace('2922', '2022')
df['Standardized_Date'] = df['Standardized_Date'].str.replace('2032', '2022')
df['Standardized_Date'] = df['Standardized_Date'].str.replace('2044', '2022')

In [ ]:
# indices_to_update = [21651, 21652, 22371, 22372] #These indexes shows date as eg: 20/04/2922 where 2922 year is not present so changing it to 2022
# correct_year = '2022'
# df.loc[indices_to_update, 'Standardized_Date'] = df.loc[indices_to_update, 'Standardized_Date'].str.replace('2922', correct_year)

In [ ]:
df[df['Standardized_Date'].isna()]

In [ ]:
df[21651:21653]

In [ ]:
df[2136:2160]

In [ ]:
df[df['Standardized_Date']=='Unknown']

In [ ]:
df['Standardized_Date'] = pd.to_datetime(df['Standardized_Date'], format='DD/MM/YYYY',errors='coerce')
df

In [ ]:
df[df['Standardized_Date'].isna()]

In [ ]:
df.to_csv('corrected_date_and_time.csv',index=False)

In [ ]:
df=pd.read_csv('corrected_date_and_time.csv')
df

In [ ]:
import pandas as pd
 
# Function to check if the value contains any non-numeric characters
def check_non_numeric(value):
    try:
        # Attempt to convert the value to a float (which handles numbers with decimals)
        float(value)
        return False  # No non-numeric characters found
    except ValueError:
        return True  # Non-numeric characters found
 
# Apply the function to the column and get a mask of rows with non-numeric values
df['non_numeric'] = df['RAW WATER FLOW IN m3/h'].apply(check_non_numeric)
 
# Filter rows that have non-numeric characters
non_numeric_rows = df[df['non_numeric']]
 
# Display rows with non-numeric values
non_numeric_rows
# non_numeric_rows.shape

# Data cleaning of RAW WATER FLOW IN m3/h column

In [ ]:
df['RAW WATER FLOW IN m3/h'].isna().sum()

In [ ]:
pd.set_option('display.max_rows', None) 
df1=df[df['RAW WATER FLOW IN m3/h'].isna()]
df1

In [ ]:
df[12104:12106]

In [ ]:
df[19072:19075]

In [ ]:
numeric_cols=['RAW WATER FLOW IN m3/h','CLEAR WATER SUMP LEVEL IN Meter', 'CLEAR WATER PUMPING FLOW m3/h',
       'TREATED WATER PRODUCTION IN m3/h']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
df['RAW WATER FLOW IN m3/h'].isna().sum()


In [ ]:
pd.set_option('display.max_rows', None) 
df2=df[df['RAW WATER FLOW IN m3/h'].isna()]
df2

In [ ]:
# new_nans_mask =new_nans_mask = (df2[df2['RAW WATER FLOW IN m3/h'].isna()]) & (df1[df1['RAW WATER FLOW IN m3/h'].notna()])

# new_nans_mask

In [ ]:
df[(df['RAW WATER FLOW IN m3/h'].isna())&(df['CLEAR WATER SUMP LEVEL IN Meter'].isna())&(df['CLEAR WATER PUMPING FLOW m3/h'].isna())&(df['TREATED WATER PRODUCTION IN m3/h']==0)]

In [ ]:
df[(df['RAW WATER FLOW IN m3/h'].isna())|(df['TREATED WATER PRODUCTION IN m3/h'].isna())]

In [ ]:
df[(df['TREATED WATER PRODUCTION IN m3/h'].isna())]

In [ ]:
df['RAW WATER FLOW IN m3/h'].nlargest(30)

In [ ]:
df['CLEAR WATER SUMP LEVEL IN Meter'].nlargest(50)

In [ ]:
indices = list(range(21220, 21230)) + \
          list(range(23200, 23201)) + \
          list(range(3310, 3320)) + \
          list(range(4560, 4570)) + \
          list(range(3475, 3485))
rows = df.loc[indices]

In [ ]:
rows